<a href="https://colab.research.google.com/github/kwb425/class2023Fall/blob/main/class2023Fall_1110.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install MySQL Server
!apt-get update
!apt-get install mysql-server
# Install MySQL python deps
!pip install mysql-connector-python
# Install FastAPI python deps
!pip install fastapi[all]
!pip install pyngrok
!pip install uvicorn

# Start the MySQL service
!service mysql start

# Log in to MySQL as root
!mysql -u root


```mysql
DROP USER IF EXISTS 'test'@'localhost';
CREATE USER 'test'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY '1234';
GRANT ALL PRIVILEGES ON *.* TO 'test'@'localhost' WITH GRANT OPTION;
FLUSH PRIVILEGES;

DROP DATABASE IF EXISTS TestDataset;
CREATE DATABASE TestDataset;
CREATE TABLE TestDataset.TestTable (
    ID INT AUTO_INCREMENT PRIMARY KEY,
    Name VARCHAR(255),
    Age INT
);
INSERT INTO TestDataset.TestTable (Name, Age) VALUES
('John', 30),
('Alice', 25),
('Bob', 35);
exit;
```

In [ ]:
# Import necessary libraries
from fastapi import FastAPI, Form, HTTPException
from fastapi.responses import HTMLResponse, JSONResponse
import mysql.connector
from pyngrok import ngrok
import time
import uvicorn
from multiprocessing import Process, Queue

# Create a Queue for inter-process communication
log_queue = Queue()

# Set up ngrok tunnel to forward traffic to port 8000 of google colab device
ngrok.set_auth_token('auth')
public_url = ngrok.connect(addr="8000")
print(f"Public URL: {public_url}")

# Create a FastAPI app
app = FastAPI()

# Function to establish a database connection
def connect_to_database():
    return mysql.connector.connect(
        host="localhost",  # Database server address
        user="test",       # Database username
        password="1234",   # User's password
        database="TestDataset"  # Name of the database
    )

# Function to fetch all records from the database
def fetch_all_records():
    try:
        conn = connect_to_database()
        cursor = conn.cursor(buffered=True)
        query = "SELECT * FROM TestDataset.TestTable"
        cursor.execute(query)
        result = cursor.fetchall()
        conn.close()
        return result
    except Exception as e:
        return []

# Handle get request to /
@app.get("/", response_class=HTMLResponse)
def read_root(name: str = None):
    records = fetch_all_records()
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>My Form</title>
        <style>
            /* Add your CSS styling here */
            body { font-family: Arial, sans-serif; color: blue; }
            input { margin: 10px; }
            .highlighted { color: red; }  /* CSS class for highlighting */
        </style>
    </head>
    <body>
        <h1>Last time...</h1>
        <form action="/submit" method="post" accept-charset="UTF-8">
            <input type="text" name="input_field" placeholder="Enter something">
            <input type="submit" value="Submit">
        </form>
        <h1>CRUD Operations</h1>
        <form action="/create" method="post" accept-charset="UTF-8">
            <input type="text" name="name" placeholder="Name">
            <input type="number" name="age" placeholder="Age">
            <input type="submit" value="Create">
        </form>
        <form action="/read" method="get">
            <input type="text" name="name" placeholder="Search by Name">
            <input type="submit" value="Read">
        </form>
        <form action="/update" method="post" accept-charset="UTF-8">
            <input type="text" name="old_name" placeholder="Old Name">
            <input type="text" name="new_name" placeholder="New Name">
            <input type="number" name="new_age" placeholder="New Age">
            <input type="submit" value="Update">
        </form>
        <form action="/delete" method="post" accept-charset="UTF-8">
            <input type="text" name="name" placeholder="Name to Delete">
            <input type="submit" value="Delete">
        </form>
        <h2>Database Records</h2>
        <ul>
    """

    for record in records:
        if name and record[1] == name:
            html_content += f'<li class="highlighted">ID: {record[0]}, Name: {record[1]}, Age: {record[2]}</li>'
        else:
            html_content += f'<li>ID: {record[0]}, Name: {record[1]}, Age: {record[2]}</li>'

    html_content += """
        </ul>
        <script>
            // Add your JavaScript here if needed
        </script>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content, media_type="text/html; charset=utf-8")


# Handle post request to /submit
@app.post("/submit")
async def submit(input_field: str = Form(...)):
    try:
        message = f"input_field: {input_field}"
        log_queue.put(message)  # Put the log message into the queue
        return JSONResponse(content={"message": message}, status_code=200)
    except Exception as e:
        return HTTPException(detail=str(e), status_code=500)

# Create a new record
@app.post("/create")
def create_record(name: str = Form(...), age: int = Form(...)):
    try:
        conn = connect_to_database()
        cursor = conn.cursor(buffered=True)
        query = "INSERT INTO TestDataset.TestTable (Name, Age) VALUES (%s, %s)"
        cursor.execute(query, (name, age))
        conn.commit()
        conn.close()
        return read_root()  # Re-render
    except Exception as e:
        return HTTPException(detail=str(e), status_code=500)

# Retrieve data
@app.get("/read")
def read_record(name: str):
    try:
        conn = connect_to_database()
        cursor = conn.cursor(buffered=True)
        query = "SELECT * FROM TestDataset.TestTable WHERE Name = %s"
        cursor.execute(query, (name,))
        result = cursor.fetchall()
        conn.close()
        return read_root(result[0][1])  # Update
    except Exception as e:
        return HTTPException(detail=str(e), status_code=500)

# Update existing data
@app.post("/update")
def update_record(old_name: str = Form(...), new_name: str = Form(...), new_age: int = Form(...)):
    try:
        conn = connect_to_database()
        cursor = conn.cursor(buffered=True)
        query = "UPDATE TzestDataset.TestTable SET Name = %s, Age = %s WHERE Name = %s"
        cursor.execute(query, (new_name, new_age, old_name))
        conn.commit()
        conn.close()
        return read_root()  # Re-render
    except Exception as e:
        return HTTPException(detail=str(e), status_code=500)

# Delete data
@app.post("/delete")
def delete_record(name: str = Form(...)):
    try:
        conn = connect_to_database()
        cursor = conn.cursor(buffered=True)
        query = "DELETE FROM TestDataset.TestTable WHERE Name = %s"
        cursor.execute(query, (name,))
        conn.commit()
        conn.close()
        return read_root()  # Re-render
    except Exception as e:
        return HTTPException(detail=str(e), status_code=500)

# Start FastAPI
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Function to print log messages from the Queue
def print_log_messages(queue):
    while True:
        message = queue.get()  # Get the log message from the queue
        print(message)

# Using multiprocessing
run_process = Process(target=run)
run_process.start()
print_process = Process(target=print_log_messages, args=(log_queue,))
print_process.start()

# Give the server a moment to start
time.sleep(3)

# Block
blocking_main_proc = input()

# Kill
run_process.terminate()
print_process.terminate()

# Double check
!kill $(pgrep ngrok)


# Handle get request to /
@app.get("/", response_class=HTMLResponse)
def read_root(name: str = None):
    records = fetch_all_records()
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <title>My Form</title>
        <style>
            /* Add your CSS styling here */
            body { font-family: Arial, sans-serif; color: blue; }
            input { margin: 10px; }
            .highlighted { color: red; }  /* CSS class for highlighting */
        </style>
    </head>
    <body>
        <h1>Last time...</h1>
        <form action="/submit" method="post" accept-charset="UTF-8">
            <input type="text" name="input_field" placeholder="Enter something">
            <input type="submit" value="Submit">
        </form>
        <h1>CRUD Operations</h1>
        <form action="/create" method="post" accept-charset="UTF-8">
            <input type="text" name="name" placeholder="Name">
            <input type="number" name="age" placeholder="Age">
            <input type="submit" value="Create">
        </form>
        <form action="/read" method="get">
            <input type="text" name="name" placeholder="Search by Name">
            <input type="submit" value="Read">
        </form>
        <form action="/update" method="post" accept-charset="UTF-8">
            <input type="text" name="old_name" placeholder="Old Name">
            <input type="text" name="new_name" placeholder="New Name">
            <input type="number" name="new_age" placeholder="New Age">
            <input type="submit" value="Update">
        </form>
        <form action="/delete" method="post" accept-charset="UTF-8">
            <input type="text" name="name" placeholder="Name to Delete">
            <input type="submit" value="Delete">
        </form>
        <h2>Database Records</h2>
        <ul>
    """

    for record in records:
        if name and record[1] == name:
            html_content += f'<li class="highlighted">ID: {record[0]}, Name: {record[1]}, Age: {record[2]}</li>'
        else:
            html_content += f'<li>ID: {record[0]}, Name: {record[1]}, Age: {record[2]}</li>'

    html_content += """
        </ul>
        <script>
            // Add your JavaScript here if needed
        </script>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content, media_type="text/html; charset=utf-8")
